In [1]:
import random
import itertools
import pandas as pd
from collections import defaultdict
import numpy as np
random.seed(123)

### b)

In [2]:
# Carregar os dados do Excel
file_path = 'Trab_Grupo.xlsx'  # Substitua pelo nome do seu arquivo Excel
sheet_name = 'Tabela'  # Substitua pelo nome da planilha, se necessário

# Carregar os dados do Excel para um DataFrame
df = pd.read_excel(file_path, sheet_name=sheet_name, header=0)

# Converter o DataFrame para uma matriz
tempos = df.values.tolist()

# Categorias dos enfermeiros
categoria_1 = [0, 1, 2, 3]
categoria_2 = [4, 5, 6, 7]
categoria_3 = [8, 9]

# Procedimentos regulares e complexos
procedimentos_regulares = [0, 1, 3, 4, 5, 8, 10, 12, 13]
procedimentos_complexos = [2, 6, 7, 9, 11]

# Períodos e procedimentos
periodos = [
    [0, 1],
    [2, 3],
    [4, 5],
    [6, 7],
    [8, 9],
    [10, 11],
    [12, 13]
]

# Função para calcular a duração de um procedimento
def calcular_duracao(procedimento, enfermeiros):
    return max(tempos[procedimento][i] for i in enfermeiros)

# Função para verificar se uma alocação é válida
def alocacao_valida(alocacao, periodo):
    # Verifica se o mesmo enfermeiro não participa em dois procedimentos no mesmo período
    enfermeiros = set()
    for proc in periodo:
        if proc in alocacao:
            for enf in alocacao[proc]:
                if enf in enfermeiros:
                    return False
                enfermeiros.add(enf)
    return True

# Função para encontrar uma alocação admissível
def encontrar_alocacao():
    while True:
        alocacao = {}
        enfermeiros_usados = {i: 0 for i in range(10)}

        for periodo in periodos:
            periodo_valido = True
            for proc in periodo:
                if proc in procedimentos_complexos:
                    categorias = categoria_2 + categoria_3
                else:
                    categorias = categoria_1 + categoria_2 + categoria_3

                # Encontrar combinação de enfermeiros válida de forma aleatória
                comb_validas = []
                for comb in itertools.combinations(categorias, 3):
                    if all(enfermeiros_usados[i] < 5 for i in comb):
                        alocacao_temporaria = alocacao.copy()
                        alocacao_temporaria[proc] = comb
                        if alocacao_valida(alocacao_temporaria, periodo):
                            comb_validas.append(comb)

                if comb_validas:
                    comb_escolhida = random.choice(comb_validas)
                    alocacao[proc] = comb_escolhida
                    for i in comb_escolhida:
                        enfermeiros_usados[i] += 1
                else:
                    # Se não houver combinações válidas para o procedimento, reiniciar a alocação
                    periodo_valido = False
                    break

            if not periodo_valido:
                break

        # Se a alocação for bem-sucedida (nenhum procedimento foi interrompido), sair do loop
        if periodo_valido and len(alocacao) == sum(len(periodo) for periodo in periodos):
            break

    return alocacao

# Função para calcular a duração total do dia de trabalho
def calcular_duracao_total(alocacao):
    duracao_total = 0
    for periodo in periodos:
        duracoes = [calcular_duracao(proc, alocacao[proc]) for proc in periodo]
        duracao_total += max(duracoes)
    return duracao_total

# Encontrar alocação admissível
alocacao = encontrar_alocacao()

# Calcular duração total do dia de trabalho
duracao_total = calcular_duracao_total(alocacao)

# Print da alocação e a duração total
print("Alocação dos enfermeiros aos procedimentos:")
for proc, enfermeiros in alocacao.items():
    print(f"P{proc + 1}: {['E' + str(i + 1) for i in enfermeiros]}")

print(f"\nDuração total do dia de trabalho: {duracao_total} minutos")

Alocação dos enfermeiros aos procedimentos:
P1: ['E1', 'E2', 'E9']
P2: ['E4', 'E5', 'E8']
P3: ['E5', 'E6', 'E9']
P4: ['E3', 'E4', 'E8']
P5: ['E1', 'E8', 'E10']
P6: ['E2', 'E4', 'E6']
P7: ['E5', 'E6', 'E8']
P8: ['E7', 'E9', 'E10']
P9: ['E3', 'E4', 'E9']
P10: ['E6', 'E8', 'E10']
P11: ['E2', 'E5', 'E10']
P12: ['E6', 'E7', 'E9']
P13: ['E1', 'E2', 'E7']
P14: ['E3', 'E4', 'E10']

Duração total do dia de trabalho: 574 minutos


### Passo 1: Definir as estruturas básicas
Vamos definir as categorias dos enfermeiros, os procedimentos, e as restrições.

In [3]:
# Carregar os dados do Excel para um DataFrame
tempos = pd.read_excel('Trab_Grupo.xlsx', header=0)

# Definição das categorias
categorias = {
    1: [1, 2, 3, 4],
    2: [5, 6, 7, 8],
    3: [9, 10]
}

# Listar todos os enfermeiros
todos_enfermeiros = categorias[1] + categorias[2] + categorias[3]

# Definição dos procedimentos complexos e regulares
procedimentos_complexos = {3, 7, 8, 10, 12}
procedimentos_regulares = set(range(1, 15)) - procedimentos_complexos

# Definir a estrutura de períodos e procedimentos
periodos = [
    (1, 2),
    (3, 4),
    (5, 6),
    (7, 8),
    (9, 10),
    (11, 12),
    (13, 14)
]

turnos = [i for i in range(1,43)]

### Passo 2: Funções para verificar restrições
Vamos criar funções para verificar se um cromossoma é válido e respeita todas as restrições.

In [4]:
def enfermeiros_validos(procedimento):
    """Retorna a lista de enfermeiros válidos para um procedimento."""
    if procedimento in procedimentos_complexos:
        return categorias[2] + categorias[3]
    else:
        return todos_enfermeiros

# Função para verificar se um cromossoma é admissível
def verifica_restricoes(cromossoma):
    contagem_procedimentos = defaultdict(int)
    enfermeiros_periodo = defaultdict(set)

    for i, enfermeiro in enumerate(cromossoma):
        procedimento = (i // 3) + 1
        periodo = (i // 6) + 1
            
        #Verifica se os procedimentos complexos são feitos por enferemriso cateogria 2 e 3 
        if procedimento in procedimentos_complexos and (enfermeiro-1)//4 == 0:
            #print("Enfermeiro",enfermeiro," para procedimento complexo",procedimento)
            return False
        
        # Verifica se o mesmo enfermeiro participa nos dois procedimentos do mesmo período
        if enfermeiro in enfermeiros_periodo[periodo]:
            #print("Mesmo enfemeiro no periodo",periodo)
            return False
        
        # Verifica se o enfermeiro já participou de 5 procedimentos
        if contagem_procedimentos[enfermeiro] >= 5:
            #print("Enfermeiro",enfermeiro, "com mais de 5 procedimentos")
            return False        
        
        contagem_procedimentos[enfermeiro] += 1
        enfermeiros_periodo[periodo].add(enfermeiro)

    # Se passar por todas as verificações, o cromossoma é admissível
    return True

### Passo 3: Função para gerar cromossomas aleatórios válidos
Vamos criar uma função para gerar um cromossoma aleatório e válido.

In [5]:
random.seed(123)
def gerar_cromossoma_valido():
    while True:
        cromossoma = []
        for procedimento in range(1, 15):
            enfermeiros = enfermeiros_validos(procedimento)
            cromossoma.extend(random.sample(enfermeiros, 3))
        if verifica_restricoes(cromossoma):
            return cromossoma

# Exemplo de uso
cromossoma_valido = gerar_cromossoma_valido()
print(cromossoma_valido)

[1, 2, 5, 4, 3, 6, 6, 5, 10, 3, 9, 7, 3, 10, 1, 7, 4, 9, 9, 5, 7, 6, 8, 10, 3, 5, 4, 10, 8, 6, 4, 1, 2, 5, 8, 7, 9, 3, 4, 2, 8, 10]


In [6]:
def print_three_by_three(vector):
    # Loop through the vector in steps of 3
    for i in range(0, len(vector), 3):
        # Slice the vector to get three elements
        chunk = vector[i:i+3]
        # Print the chunk of three elements in the desired format
        print(f"P{i//3 + 1} {chunk}")

# Example usage
print_three_by_three(cromossoma_valido)

P1 [1, 2, 5]
P2 [4, 3, 6]
P3 [6, 5, 10]
P4 [3, 9, 7]
P5 [3, 10, 1]
P6 [7, 4, 9]
P7 [9, 5, 7]
P8 [6, 8, 10]
P9 [3, 5, 4]
P10 [10, 8, 6]
P11 [4, 1, 2]
P12 [5, 8, 7]
P13 [9, 3, 4]
P14 [2, 8, 10]


### Passo 4: Calcular o Tempo Total do Cromossoma Final
Vamos criar uma função para calcular o tempo total do dia de trabalho.

In [7]:
random.seed(123)
def calcular_tempo_total(cromossoma, tempos):
    tempos_totais = []

    # Iterar sobre os períodos
    for periodo in range(7):
        tempos_periodo = []
        #print('Periodo',periodo)

        # Iterar sobre os procedimentos do período
        for procedimento in range(2):
            indice_procedimento = periodo * 2 + procedimento
            enfermeiros_procedimento = cromossoma[indice_procedimento * 3: (indice_procedimento + 1) * 3]
            # Calcular o tempo total do procedimento (usando o tempo do enfermeiro que demora mais)
            tempos_procedimento = []
            for enfermeiro in enfermeiros_procedimento:
                tempo_enfermeiro = tempos.loc[indice_procedimento][enfermeiro-1]
                tempos_procedimento.append(tempo_enfermeiro)
            tempo_total_procedimento = max(tempos_procedimento)

            tempos_periodo.append(tempo_total_procedimento)

        # Escolher o máximo dos tempos dos procedimentos no período
        tempos_totais.append(max(tempos_periodo))
        
    #Esta condicao foi adicionada por causa de uma muito rara ocasiao que a função que corrige admissibilidade não tem um enfermeiro valido para usar, logo o cromossoma fica não admissivel
    if not verifica_restricoes(cromossoma):
        #print("Não Admissivel: tempo=1000")
        return 1000
    
    return sum(tempos_totais)

# Exemplo de uso
tempo_total = calcular_tempo_total(cromossoma_valido, tempos)
print("Tempo total do cromossoma final:", tempo_total, "minutos")


Tempo total do cromossoma final: 591 minutos


### Geração da População Inicial e Cálculo do Valor de Aptidão de cada cromossoma
Criamos uma população inicial aleatória com tamanho=4 por default devido ao demorado tempo para fazer uma grande quantia de populações diferentes

In [8]:
def inicializar_populacao(tamanho_populacao=10):
    populacao = set()

    while len(populacao) < tamanho_populacao:
        cromossoma = tuple(gerar_cromossoma_valido())
        populacao.add(cromossoma)
    
    return [list(cromossoma) for cromossoma in populacao]


In [ ]:
random.seed(123)

# Inicializar a população
populacao_inicial = inicializar_populacao(20)

In [ ]:
def df_para_populacao(populacao_inicial, tempos):
    df_permutations = pd.DataFrame(columns=['cromossoma', 'tempo_total'])

    # Calculate the total time for each chromosome in the population and store it in the DataFrame
    for i, cromossoma in enumerate(populacao_inicial):
        tempo_total = calcular_tempo_total(cromossoma, tempos)
        df_permutations.loc[i] = [cromossoma, tempo_total]

    return df_permutations

df_permutations = df_para_populacao(populacao_inicial, tempos)
print(df_permutations)

### Seleção dos Pais através do Processo de Seleção por Torneio
Selecionamos os pais através da seleção por torneio, ou seja, são selecionados k cromossomas aleatorios e é escolhido o com maior aptidão (menor tempo total). 

In [ ]:
random.seed(125)
# k representa o número de cromossomas selecionados da população de forma aleatória
k = 5

# Seleção do Pai1:
pai_list1 = random.sample(list(df_permutations.index), k)
print(pai_list1)
pai1 = df_permutations.loc[pai_list1, 'tempo_total'].astype(int).idxmin()

# Seleção do Pai2: 
abc = True
while abc:
    pai_list2 = random.sample(list(df_permutations.index), k)
    print(pai_list2)
    pai2 = df_permutations.loc[pai_list2, 'tempo_total'].astype(int).idxmin() 
    if pai1 != pai2:
        abc = False
print('Pai1:',pai1) 
print('Pai2:',pai2)        

### Operação de crossover: Crossover a um ponto
Geramos 2 filhos atraves dos pais escolhidos e apagamos os 2 pais.

In [ ]:
random.seed(123)

# Selecionar número de genes num cromossoma, 3 enfermeiros por procedimento, 14 procedimentos
num_genes = 3 * 14

# Geração aleatória do número m
m = random.randint(1, num_genes - 1)
print('m:', m)

# Obter os cromossomas dos pais
df_crossover = df_permutations.loc[[pai1, pai2], :]

# Geração dos cromossomas filho
p1 = df_crossover.loc[pai1, 'cromossoma'].copy()
p2 = df_crossover.loc[pai2, 'cromossoma'].copy()
filho1 = p1.copy()
filho2 = p2.copy()

# Realizar o crossover
for i in range(m, num_genes, 1):
    filho1[i] = p2[i]
    filho2[i] = p1[i]
    
# Criar um DataFrame temporário para os filhos
df_filhos = pd.DataFrame({'cromossoma': [filho1, filho2]}, index=['filho1', 'filho2'])

# Adicionar os cromossomas dos filhos ao DataFrame df_crossover
df_crossover = pd.concat([df_crossover, df_filhos])

# Extraindo os cromossomas dos filhos
df_filhos = df_para_populacao(df_crossover.loc[['filho1', 'filho2'], 'cromossoma'].tolist(), tempos)
df_crossover.loc[['filho1', 'filho2'], 'tempo_total'] = df_filhos['tempo_total'].values

print('Cromossomas filhos após crossover:')
print(df_crossover)

### Operação Mutação: Mutação por Troca

In [ ]:
random.seed(123)

df_nextgen = df_permutations.copy()

# Mutação do filho 1
rand_1 = random.random()
print('rand_1:', rand_1)
if rand_1 <= 0.1:
    print('Mutação - filho1')
    mutants = random.sample(turnos, 2)
    print(mutants)
    p1 = df_nextgen.loc[pai1, 'cromossoma'].copy()
    print(p1)
    auxiliar = p1[mutants[0] - 1]
    p1[mutants[0] - 1] = p1[mutants[1] - 1]
    p1[mutants[1] - 1] = auxiliar
    print(p1)
    df_nextgen.at[pai1, 'cromossoma'] = p1.copy()

# Mutação do filho 2
rand_2 = random.random()
print('rand_2:', rand_2)
if rand_2 <= 0.1:
    # print('Mutação - filho2')
    mutants = random.sample(turnos, 2)
    print(mutants)
    p2 = df_nextgen.loc[pai2, 'cromossoma'].copy()
    print(p2)
    auxiliar = p2[mutants[0] - 1]
    p2[mutants[0] - 1] = p2[mutants[1] - 1]
    p2[mutants[1] - 1] = auxiliar
    print(p2)
    df_nextgen.at[pai2, 'cromossoma'] = p2.copy()

# Recalcula os custos para todos os cromossomas no final
df_filhos = df_para_populacao(df_nextgen['cromossoma'].tolist(), tempos)
df_nextgen['tempo_total'] = df_filhos['tempo_total'].values

### Algoritmo para reparar a Admissibilidade da solução

In [ ]:
random.seed(123)

turnos_complexos = [7, 8, 9, 19, 20, 21, 22, 23, 24, 28, 29, 30, 34, 35, 36]
turnos_regulares = list(set(turnos) - set(turnos_complexos))  

##print(turnos_regulares)
##print(turnos_complexos)


# Função para corrigir a admissibilidade de um cromossoma
def corrigir_admissibilidade(cromossoma):
    contagem_procedimentos = defaultdict(int)
    enfermeiros_periodo = defaultdict(set)
    
    for i in turnos_complexos:
        ##print("Turno:",i)
        enfermeiro = cromossoma[i - 1]  # Ajusta para índice baseado em zero
        procedimento = (i + 2) // 3
        ##print("Procedimento",procedimento)
        periodo = (i + 5) // 6 
        ##print("Periodo",periodo)
        
        # Verifica se os procedimentos complexos são feitos por enfermeiros das categorias 2 e 3 
        if (enfermeiro - 1) // 4 == 0 or enfermeiro in enfermeiros_periodo[periodo]:
            ##print("Correção necessária para enfermeiro regular no procedimento complexo:", enfermeiro, "no procedimento", procedimento)
            for novo_enfermeiro in random.sample(range(5, 11), 6):  # enfermeiros das categorias 2 e 3
                if novo_enfermeiro not in enfermeiros_periodo[periodo]:
                    #print("Trocar", enfermeiro, "por", novo_enfermeiro, "no procedimento complexo", procedimento)
                    cromossoma[i - 1] = novo_enfermeiro
                    contagem_procedimentos[novo_enfermeiro] += 1
                    enfermeiros_periodo[periodo].add(novo_enfermeiro)
                    break
        
        else:
            contagem_procedimentos[enfermeiro] += 1
            enfermeiros_periodo[periodo].add(enfermeiro)
    
    ##print(enfermeiros_periodo)
    ##print(contagem_procedimentos)
    
    for i in turnos_regulares:
        enfermeiro = cromossoma[i - 1]  # Ajusta para índice baseado em zero
        procedimento = (i + 2) // 3
        periodo = (i + 5) // 6
        ##print(enfermeiros_periodo[periodo])
        # Verifica se o mesmo enfermeiro participa nos dois procedimentos do mesmo período
        if enfermeiro in enfermeiros_periodo[periodo] or contagem_procedimentos[enfermeiro] >= 5:
            ##print("Correção necessária para o enfermeiro", enfermeiro, "no procedimento", procedimento)
            #print("no periodo:",enfermeiro in enfermeiros_periodo[periodo],"mais de 5:", contagem_procedimentos[enfermeiro] >= 5)
            for novo_enfermeiro in random.sample(range(1, 11),10):
                if novo_enfermeiro != enfermeiro and novo_enfermeiro not in enfermeiros_periodo[periodo] and contagem_procedimentos[novo_enfermeiro] < 5:
                    #print("Trocar", enfermeiro, "por", novo_enfermeiro, "no procedimento", procedimento)
                    cromossoma[i - 1] = novo_enfermeiro
                    contagem_procedimentos[novo_enfermeiro] += 1
                    enfermeiros_periodo[periodo].add(novo_enfermeiro)
                    break
                 
        else:
            contagem_procedimentos[enfermeiro] += 1
            enfermeiros_periodo[periodo].add(enfermeiro)       
    
    #print(cromossoma)
    ##print(contagem_procedimentos)
    return cromossoma

# Verifica e corrige a admissibilidade dos cromossomas filhos
def verificar_e_corrigir_admissibilidade(filho):
    if not verifica_restricoes(filho):
        filho1 = corrigir_admissibilidade(filho)
    return filho

In [ ]:
#Exemplo de uso
print("Filho 1 original:") 
print_three_by_three(filho1) 
print("É admissível:", verifica_restricoes(filho1))

print("\nFilho 1 corrigido:") 
filho1_corrigido = verificar_e_corrigir_admissibilidade(filho1) 
print_three_by_three(filho1_corrigido)
print("É admissível:", verifica_restricoes(filho1_corrigido))

#print("\n\nFilho 2 original:")
#print_three_by_three(filho2)
#print("É admissível:", verifica_restricoes(filho2),"\n")

#print("\nFilho 2 corrigido:")
#filho2_corrigido = verificar_e_corrigir_admissibilidade(filho2)
#print_three_by_three(filho2_corrigido)
#print("É admissível:", verifica_restricoes(filho2_corrigido))

In [ ]:
# Algoritmo trocando os filhos pelas 2 piores soluções

melhor_solucao = {
    "cromossoma": None,
    "tempo_total": float('inf')
}

# Função para atualizar a melhor solução
def atualizar_melhor_solucao(cromossoma, tempo_total):
    global melhor_solucao
    if tempo_total < melhor_solucao["tempo_total"]:
        melhor_solucao["cromossoma"] = cromossoma
        melhor_solucao["tempo_total"] = tempo_total
        print(f"Nova melhor solução encontrada: {tempo_total} minutos")
        print(f"Cromossoma: {cromossoma}")
        
# Geração de cromossomas pai e filho, e atualização da memória de longo prazo
def gerar_nova_geracao(df_permutations, tempos, k=5):
    #print_three_by_three(df_permutations)
    # Seleção do Pai1
    pai_list1 = random.sample(list(df_permutations.index), k)
    pai1 = df_permutations.loc[pai_list1, 'tempo_total'].astype(int).idxmin()

    # Seleção do Pai2
    abc = True
    while abc:
        pai_list2 = random.sample(list(df_permutations.index), k)
        pai2 = df_permutations.loc[pai_list2, 'tempo_total'].astype(int).idxmin()
        if pai1 != pai2:
            abc = False

    # Realizar o crossover
    num_genes = 3 * 14
    m = random.randint(1, num_genes - 1)
    p1 = df_permutations.loc[pai1, 'cromossoma'].copy()
    p2 = df_permutations.loc[pai2, 'cromossoma'].copy()
    filho1 = p1.copy()
    filho2 = p2.copy()

    for i in range(m, num_genes, 1):
        filho1[i] = p2[i]
        filho2[i] = p1[i]
    
    # Mutação do filho 1
    rand_1 = random.random()
    if rand_1 <= 0.1:
        mutants = random.sample(range(1, num_genes + 1), 2)
        auxiliar = filho1[mutants[0] - 1]
        filho1[mutants[0] - 1] = filho1[mutants[1] - 1]
        filho1[mutants[1] - 1] = auxiliar

    # Mutação do filho 2
    rand_2 = random.random()
    if rand_2 <= 0.1:
        mutants = random.sample(range(1, num_genes + 1), 2)
        auxiliar = filho2[mutants[0] - 1]
        filho2[mutants[0] - 1] = filho2[mutants[1] - 1]
        filho2[mutants[1] - 1] = auxiliar
    
    filho1 = verificar_e_corrigir_admissibilidade(filho1)
    filho2 = verificar_e_corrigir_admissibilidade(filho2)
    
        
    tempo_total_filho1 = calcular_tempo_total(filho1, tempos)
    tempo_total_filho2 = calcular_tempo_total(filho2, tempos)

    # Atualizar a memória de longo prazo
    atualizar_melhor_solucao(filho1, tempo_total_filho1)
    atualizar_melhor_solucao(filho2, tempo_total_filho2)

    # Substituir as duas piores soluções pelos filhos na próxima geração
    df_nextgen = df_permutations.copy()
    piores_indices = df_nextgen.sort_values(by='tempo_total', ascending=False).index[:2]
    df_nextgen.at[piores_indices[0], 'cromossoma'] = filho1.copy()
    df_nextgen.at[piores_indices[0], 'tempo_total'] = tempo_total_filho1
    df_nextgen.at[piores_indices[1], 'cromossoma'] = filho2.copy()
    df_nextgen.at[piores_indices[1], 'tempo_total'] = tempo_total_filho2
    
    #print_three_by_three(df_nextgen)
    return df_nextgen
    

In [ ]:
random.seed(123)

# Exemplo de uso das funções 
populacao_inicial = inicializar_populacao(20)
df_permutations = df_para_populacao(populacao_inicial, tempos)

In [ ]:
random.seed(123)

# Definir uma condição de parada: continuar enquanto a melhor solução for maior que 480
while melhor_solucao['tempo_total'] > 480:
    df_nextgen = gerar_nova_geracao(df_permutations, tempos)
    
    # Atualizar a população atual com a próxima geração
    df_permutations = df_nextgen.copy()

#Iterar mais 100 vezes para procurar uma solução melhor se existir
#print("\n100 iterações extra:")
for i in range(100):
    df_nextgen = gerar_nova_geracao(df_permutations, tempos)
    df_permutations = df_nextgen.copy()


# Após o loop, a melhor solução será menor ou igual a 480
print("Melhor solução final:")
#print(melhor_solucao["cromossoma"])
print_three_by_three(melhor_solucao["cromossoma"])
print(f"Tempo total: {melhor_solucao['tempo_total']} minutos")